In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from altaipony.lcio import from_mast

In [2]:
df = pd.read_csv("../results/2021_08_flares_uncorrected.csv")
old = pd.read_csv("../results/2020_02_flares_vetted.csv")
old = old[old["real?"]==1]

NumExpr defaulting to 8 threads.


In [10]:
old.groupby("ID").count().shape[0]

151

In [11]:
df.groupby("ID").count().shape[0]

152

In [4]:
# For each LC, select the latest iteration 
# of de-trending and flare finding from the table 
# and drop the rest
lateststamps = df.groupby(["ID","qcs","lc_n","mission"]).tstamp.max().reset_index()
lateststamps

,ID,qcs,lc_n,mission,tstamp
0,2MASS J01225093-2439505,3.0,0,TESS,03_08_2021_16_17_36
1,2MASS J01225093-2439505,30.0,0,TESS,09_08_2021_12_28_27
2,2MASS J02192210-3925225,3.0,0,TESS,03_08_2021_19_38_31
3,51 Eri,5.0,0,TESS,03_08_2021_21_17_20
4,AU Mic,27.0,0,TESS,09_08_2021_12_25_34
...,...,...,...,...,...
1309,WASP-99,3.0,0,TESS,03_08_2021_21_47_44
1310,WASP-99,29.0,0,TESS,09_08_2021_13_21_37
1311,WASP-99,30.0,0,TESS,09_08_2021_12_36_05
1312,YZ Cet,3.0,0,TESS,03_08_2021_11_00_00


In [54]:
# merge in the full table such that we are left wit only those latest iterations
lastdf = lateststamps.merge(df, how="left", on=["ID","qcs","lc_n","mission","tstamp"])

In [56]:
lastdf.shape
lastdf.head()

(4227, 17)

In [59]:
lastdf.sort_values(by=["ID","tstamp"]).to_csv("../results/2021_08_flares.csv")

# obsdurs table vetting

In [23]:
obs = pd.read_csv("../results/2021_08_obsdurs.csv")

In [24]:
obs.shape

(2455, 6)

In [25]:
obs = obs.drop_duplicates(subset=["ID","qcs","lc_n","mission"])[["ID","qcs","lc_n","mission"]]
obs["version"] = "new"

In [26]:
obs.shape

(1377, 5)

In [27]:
obsold  = pd.read_csv("../results/2020_02_obsdurs.csv")

In [28]:
obsold = obsold.drop_duplicates(subset=["ID","qcs","lc_n","mission"])[["ID","qcs","lc_n","mission"]]
obsold["version"] = "old"

In [29]:
obsold.shape

(1327, 5)

In [30]:
merg = obs.merge(obsold, on=["ID","qcs","lc_n","mission"], how="outer", suffixes=("_new","_old"))
merg.groupby(["version_new","version_old"]).count()

,,ID,qcs,lc_n,mission
version_new,version_old,,,,
new,old,1327,1327,1327,1327


In [31]:
merg[merg.version_new.isnull()].sort_values("ID")#.to_csv("../results/leftoverlcs.csv", index=False)

,ID,qcs,lc_n,mission,version_new,version_old


In [33]:
merg[merg.version_old.isnull()].sort_values("ID").shape

(50, 6)